Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
Результат классификации запишите в столбец class

Уточнения: 
1) Не понял, про какие материалы из занятия шла речь в задании, поэтому сделал на основе датасета из прошлого ДЗ
2) интервал 4 < и > 4.5 выпадал из условий задания, поэтому я решил его отнести к "высокому рейтингу"

In [1]:
import pandas as pd
data = pd.read_csv('ratings.csv')
movie_name = pd.read_csv('movies.csv')

In [61]:
def check_rating(rating):
    '''Функция возвращает категорию рейтинга в зависимости от его величины
    '''
    class_rating = ["низкий рейтинг", "средний рейтинг", "низкий рейтинг"]
    if rating <= 2:
        return class_rating[0]
    elif rating <= 4:
        return class_rating[1]
    elif rating > 4:
        return class_rating[2]

data_clear = data.groupby('movieId')['rating'].mean().reset_index()
data_clear['class'] = data_clear['rating'].apply(lambda rating: check_rating(rating))
data_clear = pd.merge(movie_name, data_clear , on = 'movieId', how = 'left')[['title', 'rating', 'class']]
data_clear.head()

,title,rating,class
0,Toy Story (1995),3.886649,средний рейтинг
1,Jumanji (1995),3.246583,средний рейтинг
2,Grumpier Old Men (1995),3.173981,средний рейтинг
3,Waiting to Exhale (1995),2.874540,средний рейтинг
4,Father of the Bride Part II (1995),3.077291,средний рейтинг


Задание 2
Используем файл keywords.csv.
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
Результат классификации запишите в отдельный столбец region.

In [3]:
keyword = pd.read_csv('keywords.csv')

In [4]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
def check_region(word):
    '''Присваивает регион для запросов, в которых есть город из списка
    '''
    for key, value in geo_data.items():
        if word.lower() in value:
            return key
            break
    return 'undefined'
keyword['region'] = keyword['keyword'].apply(lambda word: check_region(word))
keyword[keyword['region'] != 'undefined'].head()

,keyword,shows,region
1259,москва,153050,Центр
5858,владивосток,43038,Дальний Восток
6075,ярославль,41720,Центр
8977,мурманск,29984,Северо-Запад
9084,хабаровск,29655,Дальний Восток


Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
1 В переменную years запишите список из всех годов с 1950 по 2010.
2 Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
2.1 для каждой строки пройдите по всем годам списка years
2.2 если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
2.3 если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [195]:
import re
def production_year(title):
    '''Функция достаёт из названия фильма его год выпуска
    '''
    pattern = r'(?:\()(\d{4})(?:\))$'
    year = re.findall(pattern,title)
    for val in year:   #костыль, так как возврат по индексу [0] ничего не дал
        return val

data_clear['year'] = data_clear['title'].apply(lambda x: production_year(x))
if data_clear['year'].isna().sum() != 0: #проверка отсутствующих значений
     data_clear['year'] = data_clear['year'].fillna('1900')
data_year = data_clear.groupby('year')['rating'].mean().reset_index()
data_year.sort_values('rating', ascending=False).head(10)

,year,rating
26,1911,3.379167
39,1924,3.357279
43,1928,3.352209
41,1926,3.331629
38,1923,3.297808
78,1963,3.257071
34,1919,3.255699
40,1925,3.254458
77,1962,3.240344
76,1961,3.228740
